In [0]:
%pip install databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
%run "../Configuraciones/variables_configuraciones"

In [0]:
print(RUTA_CARPETA_GIT)
print(NOMBRE_JOB_FLUJO_BRONCE_ORO)
print(NOTIFICACIONES_JOB_FLUJO_BRONCE_ORO)

In [0]:
ruta_carpeta_git = RUTA_CARPETA_GIT
nombre_job = NOMBRE_JOB_FLUJO_BRONCE_ORO
notificaciones = NOTIFICACIONES_JOB_FLUJO_BRONCE_ORO

In [0]:
job_config =    {
        "email_notifications": {
            "no_alert_for_skipped_runs": False,
            "on_failure": notificaciones,
            "on_start": notificaciones,
            "on_success": notificaciones
        },
        "format": "MULTI_TASK",
        "max_concurrent_runs": 1,
        "name": nombre_job,
        "notification_settings": {
            "no_alert_for_canceled_runs": False,
            "no_alert_for_skipped_runs": False
        },
        "performance_target": "PERFORMANCE_OPTIMIZED",
        "queue": {
            "enabled": True
        },
        "tasks": [
            {
                "email_notifications": {},
                "notebook_task": {
                    "notebook_path": f"{ruta_carpeta_git}/Databricks/Porcesos/1_carga_datos_vehiculos",
                    "source": "WORKSPACE"
                },
                "run_if": "ALL_SUCCESS",
                "task_key": "Cragar_de_bronce_a_plata",
                "timeout_seconds": 0,
                "webhook_notifications": {}
            },
            {
                "depends_on": [
                    {
                        "task_key": "Cragar_de_bronce_a_plata"
                    }
                ],
                "email_notifications": {},
                "notebook_task": {
                    "notebook_path": f"{ruta_carpeta_git}/Databricks/Porcesos/3_Proceso_Gold_Vehiculos",
                    "source": "WORKSPACE"
                },
                "run_if": "ALL_SUCCESS",
                "task_key": "carga_capa_oro_vehiculos",
                "timeout_seconds": 0,
                "webhook_notifications": {}
            },
            {
                "depends_on": [
                    {
                        "task_key": "Cragar_de_bronce_a_plata"
                    }
                ],
                "email_notifications": {},
                "notebook_task": {
                    "notebook_path": f"{ruta_carpeta_git}/Databricks/Porcesos/2_Proceso_Gold_Canton",
                    "source": "WORKSPACE"
                },
                "run_if": "ALL_SUCCESS",
                "task_key": "carga_dimension_canton",
                "timeout_seconds": 0,
                "webhook_notifications": {}
            },
            {
                "depends_on": [
                    {
                        "task_key": "carga_capa_oro_vehiculos"
                    }
                ],
                "email_notifications": {},
                "notebook_task": {
                    "notebook_path": f"{ruta_carpeta_git}/Databricks/Porcesos/4_Proceso_Gold_Fecha",
                    "source": "WORKSPACE"
                },
                "run_if": "ALL_SUCCESS",
                "task_key": "carga_dimension_fecha",
                "timeout_seconds": 0,
                "webhook_notifications": {}
            }
        ],
        "timeout_seconds": 0,
        "webhook_notifications": {}
    }

In [0]:
from databricks.sdk.service.jobs import JobSettings as Job
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
new_config = Job.from_dict(job_config)
new_job = w.jobs.create(**new_config.as_shallow_dict())

### JobId:

In [0]:
print("Nuevo job creado con ID:", new_job.job_id)